# Extract Soil Data
Parses all the soil survey areas in Arizona and extract the soil data in tabular format

In [1]:
import sys
base_path = "src/"
if base_path not in sys.path:
    sys.path.append(base_path)

In [2]:
import pathlib

import pandas as pd

from dataloaders import (
    load_area_legend_db, 
    load_chorizon_db, 
    load_component_db, 
    load_mapunit_db
)

We need to add the areaname and legend to each table

In [3]:
base_dir = "./datasets/arizona/"
area_symbols = [folder.stem for folder in pathlib.Path(base_dir).iterdir() if folder.is_dir()]
chorizon__table_filename_suffix = "chorizon.txt"
area_legend_table_filename_suffix = "legend.txt"
component_table_filename_suffix = "comp.txt"
mapunit_table_filename_suffix = "mapunit.txt"

We need to left join the mapunit table to the component table. 
We then need to left join this table to the chorizon table.
Finally we need to filter the table to the columns we need.
Then we save

In [8]:
target_column_names =[
    "mukey",
    "muname",
    "musym",
    "farmlndcl",
    "cokey",
    "compname",
    "comppct_r",
    "desgnmaster",
    "chkey",
    "sandtotal_l",
    "sandtotal_r",
    "sandtotal_h",
    "silttotal_l",
    "silttotal_r",
    "silttotal_h",
    "claytotal_l",
    "claytotal_r",
    "claytotal_h",
    "ph1to1h2o_l",
    "ph1to1h2o_r",
    "ph1to1h2o_h",
    "ksat_l",
    "ksat_r",
    "ksat_h"
]

In [5]:
final_columnname_remappings = {
    "mukey": "mapunit_key",
    "muname": "mapunit_name",
    "musymbol": "mapunit_symbol",
    "farmlndcl": "farmland_status",
    "cokey": "component_key",
    "compname": "component_name",
    "comppct_r": "component_percent_of_mapunit",
    "chkey": "chorizon_key",
    "desgnmaster": "master_soil_horizon_id",
    "sandtotal_l": "frac_weight_sand_layer_low",
    "sandtotal_r": "frac_weight_sand_layer_reg",
    "sandtotal_h": "frac_weight_sand_layer_high",
    "silttotal_l": "frac_weight_silt_layer_low",
    "silttotal_r": "frac_weight_silt_layer_reg",
    "silttotal_h": "frac_weight_silt_layer_high",
    "claytotal_l": "frac_weight_clay_layer_low",
    "claytotal_r": "frac_weight_clay_layer_reg",
    "claytotal_h": "frac_weight_clay_layer_high",
    "ph1to1h2o_l": "ph_layer_low",
    "ph1to1h2o_r": "ph_layer_reg",
    "ph1to1h2o_h": "ph_layer_high",
    "ksat_l": "amount_water_movement_low",
    "ksat_r": "amount_water_movement_reg",
    "ksat_h": "amount_water_movement_high"
}

### Test Case

In [8]:
area_dir = base_dir + "AZ701/tabular/"

In [19]:
component_df = load_component_db(area_dir + component_table_filename_suffix)
chorizon_df = load_chorizon_db(area_dir + chorizon__table_filename_suffix)
legend_df = load_area_legend_db(area_dir + area_legend_table_filename_suffix)
mapunit_df = load_mapunit_db(area_dir + mapunit_table_filename_suffix)

In [20]:
component_df

,comppct_l,comppct_r,comppct_h,compname,compkind,majcompflag,otherph,localphase,slope_l,slope_r,slope_h,slopelenusle_l,slopelenusle_r,slopelenusle_h,runoff,tfact,wei,weg,erocl,earthcovkind1,earthcovkind2,hydricon,hydricrating,drainagecl,elev_l,elev_r,elev_h,aspectccwise,aspectrep,aspectcwise,geomdesc,albedodry_l,albedodry_r,albedodry_h,airtempa_l,airtempa_r,airtempa_h,map_l,map_r,map_h,...,wlwoodland,wlwetland,soilslippot,frostact,initsub_l,initsub_r,initsub_h,totalsub_l,totalsub_r,totalsub_h,hydgrp,corcon,corsteel,taxciname,taxorder,taxsuborder,taxgrtgroup,taxsubgrp,taxpartsize,taxpartsizemod,taxceactcl,taxreaction,taxtempcl,taxmoistscl,taxtempregime,soiltaxedition,castorieindex,flecolcomnum,flhe,flphe,flsoilleachpot,flsoirunoffpot,fltemik2use,fltriumph2use,indraingrp,innitrateleachi,misoimgmtgrp,vasoimgtgrp,mukey,cokey
0,NaN,40,NaN,Udic Haplustolls,Taxon above family,Yes,NaN,NaN,15.0,35.0,55.0,3.0,5.0,6.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,2286.0,2438.0,2591.0,360.0,0.0,0.0,plateaus,NaN,0.16,NaN,2.0,4.5,7.0,635.0,737.0,838.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,D,NaN,NaN,Udic Haplustolls,Mollisols,Ustolls,Haplustolls,Udic Haplustolls,NaN,NaN,NaN,NaN,NaN,Udic,frigid,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,615954,25724121
1,NaN,41,NaN,Lithic Haplustolls,Taxon above family,Yes,NaN,NaN,15.0,35.0,55.0,3.0,5.0,6.0,Very high,1.0,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,2134.0,2362.5,2591.0,360.0,0.0,0.0,plateaus,NaN,0.16,NaN,2.0,4.5,7.0,635.0,737.0,838.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,D,NaN,NaN,Lithic Haplustolls,Mollisols,Ustolls,Haplustolls,Lithic Haplustolls,NaN,NaN,NaN,NaN,NaN,Udic,frigid,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,615954,25724122
2,NaN,19,NaN,Rock outcrop,Miscellaneous area,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,615954,25724123
3,NaN,20,NaN,Lithic Calciargids,Taxon above family,Yes,NaN,NaN,2.0,8.0,15.0,3.0,17.0,30.0,Medium,1.0,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,366.0,427.0,488.0,360.0,0.0,0.0,plateaus,NaN,0.30,NaN,21.1,22.5,24.0,76.0,114.0,152.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,D,NaN,NaN,Lithic Calciargids,Aridisols,Argids,Calciargids,Lithic Calciargids,NaN,NaN,NaN,NaN,NaN,Typic,hyperthermic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,615955,25724124
4,NaN,80,NaN,Lithic Torriorthents,Taxon above family,Yes,NaN,NaN,2.0,28.0,55.0,3.0,5.0,6.0,Very high,1.0,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,366.0,427.0,488.0,360.0,0.0,0.0,plateaus,NaN,0.37,NaN,21.1,22.5,24.0,76.0,114.0,152.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,D,NaN,NaN,Lithic Torriorthents,Entisols,Orthents,Torriorthents,Lithic Torriorthents,NaN,NaN,NaN,NaN,NaN,NaN,hyperthermic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,615955,25724125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447,NaN,40,NaN,Valleycity family,Family,Yes,NaN,NaN,8.0,12.0,15.0,3.0,17.0,30.0,High,1.0,48.0,6,Class 1,NaN,NaN,NaN,No,Well drained,1372.0,1448.0,1524.0,360.0,0.0,0.0,plateaus,NaN,0.23,NaN,13.0,13.5,13.9,152.0,204.0,254.0,...,NaN,NaN,NaN,Moderate,0.0,NaN,0.0,NaN,NaN,NaN,D,Low,Moderate,"Loamy-skeletal, mixed, superactive, mesic Lith...",Aridisols,Argids,Haplargids,Lithic Haplargids,loamy-skeletal,not used,superactive,not used,mesic,Typic,mesic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,616118,25724401
448,NaN,5,NaN,Unnamed soils,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,N

In [21]:
mapunit_df

,musym,muname,mukind,mustatus,muacres,mapunitlfw_l,mapunitlfw_r,mapunitlfw_h,mapunitpfa_l,mapunitpfa_r,mapunitpfa_h,farmlndcl,muhelcl,muwathelcl,muwndhelcl,interpfocus,invesintens,iacornsr,nhiforsoigrp,nhspiagr,vtsepticsyscl,mucertstat,lkey,mukey
0,1,"Albers clay loam, 0 to 1 percent slopes",Consociation,NaN,580,NaN,NaN,NaN,NaN,NaN,NaN,Not prime farmland,NaN,NaN,NaN,NaN,Order 3,NaN,NaN,NaN,NaN,NaN,10290,615792
1,2,"Argic Petrocalcids, 8 to 15 percent slopes",Consociation,NaN,1578,NaN,NaN,NaN,NaN,NaN,NaN,Not prime farmland,NaN,NaN,NaN,NaN,Order 4,NaN,NaN,NaN,NaN,NaN,10290,615793
2,3,"Argic Petrocalcids, warm, 2 to 30 percent slopes",Consociation,NaN,758,NaN,NaN,NaN,NaN,NaN,NaN,Not prime farmland,NaN,NaN,NaN,NaN,Order 4,NaN,NaN,NaN,NaN,NaN,10290,615794
3,4,"Aridic Haplustalfs-Lithic Haplustalfs complex,...",Complex,NaN,829,NaN,NaN,NaN,NaN,NaN,NaN,Not prime farmland,NaN,NaN,NaN,NaN,Order 4,NaN,NaN,NaN,NaN,NaN,10290,615795
4,5,"Aridic Haplustepts, 0 to 8 percent slopes",Consociation,NaN,71,NaN,NaN,NaN,NaN,NaN,NaN,Not prime farmland,NaN,NaN,NaN,NaN,Order 4,NaN,NaN,NaN,NaN,NaN,10290,615796
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172,175,"Yumtheska-Toqui-Rock outcrop complex, 2 to 8 p...",Complex,NaN,3693,NaN,NaN,NaN,NaN,NaN,NaN,Not prime farmland,NaN,NaN,NaN,NaN,Order 3,NaN,NaN,NaN,NaN,NaN,10290,616067
173,176,"Yumtheska-Toqui-Rock outcrop complex, 15 to 40...",Complex,NaN,2795,NaN,NaN,NaN,NaN,NaN,NaN,Not prime farmland,NaN,NaN,NaN,NaN,Order 3,NaN,NaN,NaN,NaN,NaN,10290,616068
174,177,"Zibate family, 2 to 8 percent slopes",Consociation,NaN,447,NaN,NaN,NaN,NaN,NaN,NaN,Not prime farmland,NaN,NaN,NaN,NaN,Order 3,NaN,NaN,NaN,NaN,NaN,10290,616069
175,159,"Valleycity-Berzatic-Seeg families complex, 8 t...",Complex,NaN,1325,NaN,NaN,NaN,NaN,NaN,NaN,Not prime farmland,NaN,NaN,NaN,NaN,Order 3,NaN,NaN,NaN,NaN,NaN,10290,616118


In [24]:
component_merged_df = pd.merge(component_df, mapunit_df, on='mukey', how='left')
assert len(component_merged_df) == len(component_df)

In [25]:
component_merged_df.head()

,comppct_l,comppct_r,comppct_h,compname,compkind,majcompflag,otherph,localphase,slope_l,slope_r,slope_h,slopelenusle_l,slopelenusle_r,slopelenusle_h,runoff,tfact,wei,weg,erocl,earthcovkind1,earthcovkind2,hydricon,hydricrating,drainagecl,elev_l,elev_r,elev_h,aspectccwise,aspectrep,aspectcwise,geomdesc,albedodry_l,albedodry_r,albedodry_h,airtempa_l,airtempa_r,airtempa_h,map_l,map_r,map_h,...,taxmoistscl,taxtempregime,soiltaxedition,castorieindex,flecolcomnum,flhe,flphe,flsoilleachpot,flsoirunoffpot,fltemik2use,fltriumph2use,indraingrp,innitrateleachi,misoimgmtgrp,vasoimgtgrp,mukey,cokey,musym,muname,mukind,mustatus,muacres,mapunitlfw_l,mapunitlfw_r,mapunitlfw_h,mapunitpfa_l,mapunitpfa_r,mapunitpfa_h,farmlndcl,muhelcl,muwathelcl,muwndhelcl,interpfocus,invesintens,iacornsr,nhiforsoigrp,nhspiagr,vtsepticsyscl,mucertstat,lkey
0,NaN,40,NaN,Udic Haplustolls,Taxon above family,Yes,NaN,NaN,15.0,35.0,55.0,3.0,5.0,6.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,2286.0,2438.0,2591.0,360.0,0.0,0.0,plateaus,NaN,0.16,NaN,2.0,4.5,7.0,635.0,737.0,838.0,...,Udic,frigid,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,615954,25724121,65,Lithic Haplustolls-Udic Haplustolls-Rock outcr...,Complex,NaN,15412,NaN,NaN,NaN,NaN,NaN,NaN,Not prime farmland,NaN,NaN,NaN,NaN,Order 4,NaN,NaN,NaN,NaN,NaN,10290
1,NaN,41,NaN,Lithic Haplustolls,Taxon above family,Yes,NaN,NaN,15.0,35.0,55.0,3.0,5.0,6.0,Very high,1.0,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,2134.0,2362.5,2591.0,360.0,0.0,0.0,plateaus,NaN,0.16,NaN,2.0,4.5,7.0,635.0,737.0,838.0,...,Udic,frigid,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,615954,25724122,65,Lithic Haplustolls-Udic Haplustolls-Rock outcr...,Complex,NaN,15412,NaN,NaN,NaN,NaN,NaN,NaN,Not prime farmland,NaN,NaN,NaN,NaN,Order 4,NaN,NaN,NaN,NaN,NaN,10290
2,NaN,19,NaN,Rock outcrop,Miscellaneous area,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,615954,25724123,65,Lithic Haplustolls-Udic Haplustolls-Rock outcr...,Complex,NaN,15412,NaN,NaN,NaN,NaN,NaN,NaN,Not prime farmland,NaN,NaN,NaN,NaN,Order 4,NaN,NaN,NaN,NaN,NaN,10290
3,NaN,20,NaN,Lithic Calciargids,Taxon above family,Yes,NaN,NaN,2.0,8.0,15.0,3.0,17.0,30.0,Medium,1.0,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,366.0,427.0,488.0,360.0,0.0,0.0,plateaus,NaN,0.30,NaN,21.1,22.5,24.0,76.0,114.0,152.0,...,Typic,hyperthermic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,615955,25724124,66,Lithic Torriorthents-Lithic Calciargids comple...,Complex,NaN,3366,NaN,NaN,NaN,NaN,NaN,NaN,Not prime farmland,NaN,NaN,NaN,NaN,Order 4,NaN,NaN,NaN,NaN,NaN,10290
4,NaN,80,NaN,Lithic Torriorthents,Taxon above family,Yes,NaN,NaN,2.0,28.0,55.0,3.0,5.0,6.0,Very high,1.0,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,366.0,427.0,488.0,360.0,0.0,0.0,plateaus,NaN,0.37,NaN,21.1,22.5,24.0,76.0,114.0,152.0,...,NaN,hyperthermic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,615955,25724125,66,Lithic Torriorthents-Lithic Calciargids comple...,Complex,NaN,3366,NaN,NaN,NaN,NaN,NaN,NaN,Not prime farmland,NaN,NaN,NaN,NaN,Order 4,NaN,NaN,NaN,NaN,NaN,10290


In [27]:
chorizon_merged_df = pd.merge(chorizon_df, component_merged_df, on='cokey', how='left')
assert len(chorizon_df) == len(chorizon_merged_df)

In [29]:
chorizon_merged_df.head()

,hzname,desgndisc,desgnmaster,desgnmasterprime,desgnvert,hzdept_l,hzdept_r,hzdept_h,hzdepb_l,hzdepb_r,hzdepb_h,hzthk_l,hzthk_r,hzthk_h,fraggt10_l,fraggt10_r,fraggt10_h,frag3to10_l,frag3to10_r,frag3to10_h,sieveno4_l,sieveno4_r,sieveno4_h,sieveno10_l,sieveno10_r,sieveno10_h,sieveno40_l,sieveno40_r,sieveno40_h,sieveno200_l,sieveno200_r,sieveno200_h,sandtotal_l,sandtotal_r,sandtotal_h,sandvc_l,sandvc_r,sandvc_h,sandco_l,sandco_r,...,taxtempcl,taxmoistscl,taxtempregime,soiltaxedition,castorieindex,flecolcomnum,flhe,flphe,flsoilleachpot,flsoirunoffpot,fltemik2use,fltriumph2use,indraingrp,innitrateleachi,misoimgmtgrp,vasoimgtgrp,mukey,musym,muname,mukind,mustatus,muacres,mapunitlfw_l,mapunitlfw_r,mapunitlfw_h,mapunitpfa_l,mapunitpfa_r,mapunitpfa_h,farmlndcl,muhelcl,muwathelcl,muwndhelcl,interpfocus,invesintens,iacornsr,nhiforsoigrp,nhspiagr,vtsepticsyscl,mucertstat,lkey
0,3Ab,NaN,A,NaN,NaN,NaN,127,NaN,NaN,152,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,95.0,97.5,100.0,95.0,97.5,100.0,85.0,90.0,95.0,60.0,67.5,75.0,42.1,42.1,42.1,NaN,3.9,NaN,NaN,5.5,...,mesic,Aridic (torric),mesic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,615967,78,"Lykorly loam, 2 to 4 percent slopes",Consociation,NaN,3075,NaN,NaN,NaN,NaN,NaN,NaN,Not prime farmland,NaN,NaN,NaN,NaN,Order 3,NaN,NaN,NaN,NaN,NaN,10290
1,A2,NaN,A,NaN,NaN,NaN,10,NaN,NaN,20,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,95.0,97.5,100.0,95.0,97.5,100.0,85.0,90.0,95.0,60.0,67.5,75.0,42.1,42.1,42.1,NaN,3.9,NaN,NaN,5.5,...,mesic,Aridic (torric),mesic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,615967,78,"Lykorly loam, 2 to 4 percent slopes",Consociation,NaN,3075,NaN,NaN,NaN,NaN,NaN,NaN,Not prime farmland,NaN,NaN,NaN,NaN,Order 3,NaN,NaN,NaN,NaN,NaN,10290
2,Bw1,NaN,B,NaN,NaN,NaN,20,NaN,NaN,51,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,95.0,97.5,100.0,95.0,97.5,100.0,85.0,90.0,95.0,60.0,67.5,75.0,42.1,42.1,42.1,NaN,3.9,NaN,NaN,5.5,...,mesic,Aridic (torric),mesic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,615967,78,"Lykorly loam, 2 to 4 percent slopes",Consociation,NaN,3075,NaN,NaN,NaN,NaN,NaN,NaN,Not prime farmland,NaN,NaN,NaN,NaN,Order 3,NaN,NaN,NaN,NaN,NaN,10290
3,2Btk1,NaN,B,NaN,NaN,NaN,79,NaN,NaN,102,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,95.0,97.5,100.0,95.0,97.5,100.0,90.0,95.0,100.0,85.0,90.0,95.0,16.9,16.9,16.9,NaN,0.1,NaN,NaN,0.4,...,mesic,Aridic (torric),mesic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,615967,78,"Lykorly loam, 2 to 4 percent slopes",Consociation,NaN,3075,NaN,NaN,NaN,NaN,NaN,NaN,Not prime farmland,NaN,NaN,NaN,NaN,Order 3,NaN,NaN,NaN,NaN,NaN,10290
4,2Btk2,NaN,B,NaN,NaN,NaN,102,NaN,NaN,127,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,85.0,92.5,100.0,80.0,90.0,100.0,75.0,85.0,95.0,70.0,82.5,95.0,16.9,16.9,16.9,NaN,0.1,NaN,NaN,0.4,...,mesic,Aridic (torric),mesic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,615967,78,"Lykorly loam, 2 to 4 percent slopes",Consociation,NaN,3075,NaN,NaN,NaN,NaN,NaN,NaN,Not prime farmland,NaN,NaN,NaN,NaN,Order 3,NaN,NaN,NaN,NaN,NaN,10290


In [62]:
chorizon_merged_df["areaname"] = legend_df.areaname.unique()[0]
chorizon_merged_df["areasymbol"] = legend_df.areasymbol.unique()[0]
output_df = chorizon_merged_df[target_column_names].rename(columns=final_columnname_remappings)
output_df.shape

# Process All Arizona

In [15]:
def process_all(base_dir: str, area_symbols: list[str], target_column_names: list[str]) -> pd.DataFrame:
    final_df = None
    for symbol in area_symbols:
        print(f"Processing area with symbol {symbol}", end="...")
        area_dir = pathlib.Path(base_dir).joinpath(symbol).joinpath("tabular")
        component_df = load_component_db(area_dir.joinpath(component_table_filename_suffix))
        chorizon_df = load_chorizon_db(area_dir.joinpath(chorizon__table_filename_suffix))
        legend_df = load_area_legend_db(area_dir.joinpath(area_legend_table_filename_suffix))
        mapunit_df = load_mapunit_db(area_dir.joinpath(mapunit_table_filename_suffix))
        if len(chorizon_df) < 1:
            print(f"Skipping {symbol} as no horizon data", end="\n")
            continue

        print(f"Found {len(chorizon_df)} rows in horizon dataset", end="\n")
        component_merged_df = pd.merge(component_df, mapunit_df, on='mukey', how='left')
        chorizon_merged_df = pd.merge(chorizon_df, component_merged_df, on='cokey', how='left')
        output_df = chorizon_merged_df[target_column_names].copy()
        output_df["areaname"] = legend_df.areaname.unique()[0]
        output_df["areasymbol"] = legend_df.areasymbol.unique()[0]
        if final_df is None:
            final_df = output_df
        else:
            final_df = pd.concat([final_df, output_df])

    return final_df

In [16]:
output_df = process_all(base_dir, area_symbols, target_column_names)
output_df = output_df.rename(columns=final_columnname_remappings)
output_df.to_csv("./mapped_soil_data.csv", index=False)

Processing area with symbol AZ633...Found 259 rows in horizon dataset
Processing area with symbol AZ658...Found 164 rows in horizon dataset
Processing area with symbol AZ693...Found 176 rows in horizon dataset
Processing area with symbol AZ667...Found 351 rows in horizon dataset
Processing area with symbol AZ669...Found 461 rows in horizon dataset
Processing area with symbol AZ656...Found 163 rows in horizon dataset
Processing area with symbol AZ651...Found 552 rows in horizon dataset
Processing area with symbol AZ635...Found 422 rows in horizon dataset
Processing area with symbol AZ668...Found 260 rows in horizon dataset
Processing area with symbol AZ657...Found 849 rows in horizon dataset
Processing area with symbol AZ695...Found 843 rows in horizon dataset
Processing area with symbol AZ661...Found 672 rows in horizon dataset
Processing area with symbol AZ659...Found 167 rows in horizon dataset
Processing area with symbol AZ666...Found 628 rows in horizon dataset
Processing area with

In [17]:
output_df.shape

(24441, 26)